In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'gender_submission.csv', 'test.csv']


Predicting the survival of the passenger using the given data set has to be treated as "Classification" problem. Reading training data set

In [2]:
model = pd.read_csv('../input/train.csv',index_col=0)
model.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Data set has lot of information, we need to cleanup unwanted information and prepare proper data set to train model. Availibility of irrelevant informations will introduce noise to during the training. For example, Fields like Ticket, PassengerId were random numbers which is of no improtance in predicitng the survival classification.

In further steps data set has to cleaned up to the best possible extent to make sure all noisy data is eliminated. This process is called Data Wrangling. 'pandas' is one of the best available PYTHON libraries to perform this task effectively.

In [3]:
model = model[['Pclass','Sex','Age','SibSp','Parch','Fare','Survived']]
model.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
PassengerId,,,,,,,
1,3,male,22.0,1,0,7.2500,0
2,1,female,38.0,1,0,71.2833,1
3,3,female,26.0,0,0,7.9250,1
4,1,female,35.0,1,0,53.1000,1
5,3,male,35.0,0,0,8.0500,0


It is always more efficient to process with numerics than text. Normalising test to binary values is considered as state-of-art data wrangling technique.

In [4]:
model['Sex'] = model['Sex'].map({'male':0, 'female':1})

model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 7 columns):
Pclass      891 non-null int64
Sex         891 non-null int64
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Survived    891 non-null int64
dtypes: float64(2), int64(5)
memory usage: 55.7 KB


Filling NaN with the mean of the column data. NaN must be eliminated either by filling with mean of the column data or by dropping NaN data.

In [5]:
model["Age"].fillna(model["Age"].mean(), inplace= True)
model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 7 columns):
Pclass      891 non-null int64
Sex         891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Survived    891 non-null int64
dtypes: float64(2), int64(5)
memory usage: 55.7 KB


Dropping 'Survived' result from model and treating it as expected prediction result.

In [6]:
x_train = model.drop('Survived', axis = 1)
y_train = model['Survived']
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,,,,,,
1,3,0,22.0,1,0,7.2500
2,1,1,38.0,1,0,71.2833
3,3,1,26.0,0,0,7.9250
4,1,1,35.0,1,0,53.1000
5,3,0,35.0,0,0,8.0500


Splits the data set to training and test data set

In [7]:
#from sklearn.model_selection import train_test_split
#x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 1)

Instantiating Decission Tree Classifier. One of the efficient algorithm for Supervied Learning Classification Problems. It is highly recommended to try out "Support Vector Classifier" and compare the accuracy score in order to decide which is performing better.

In [8]:
from sklearn import tree
result = tree.DecisionTreeClassifier()

In [9]:
from sklearn import tree
result = tree.DecisionTreeClassifier(min_samples_split = 15)

Training the model. Entire training data set is used to train model. Same datashould not be used for testing. Prepare separate set of test data which the trained model is unaware of. This approach yeilds better prediction results for different kinds of test cases.

In [10]:
result.fit(x_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=15,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Loading fresh test data set

In [11]:
test = pd.read_csv('../input/test.csv')


Wrangling test data set to fit resemble to training data structure.

In [12]:
test = test[['Pclass','Sex','Age','SibSp','Parch','Fare']]


In [13]:
test["Age"].fillna(test["Age"].mean(), inplace= True)


In [14]:
test["Sex"] = test["Sex"].map({'male':0,'female':1})
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,0,34.5,0,0,7.8292
1,3,1,47.0,1,0,7.0000
2,2,0,62.0,0,0,9.6875
3,3,0,27.0,0,0,8.6625
4,3,1,22.0,1,1,12.2875


In [15]:

test["Fare"].fillna(test["Fare"].mean(), inplace= True)


Ideal Predictions result to eveluate test data prediction accuracy

In [16]:
survial = pd.read_csv('../input/gender_submission.csv',index_col=0)


Predict and evaluate accuracy against test data set

In [17]:
survival_prediction = result.predict(test)
from sklearn.metrics import accuracy_score
accuracy_score(survial,survival_prediction)

0.8516746411483254